In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def plus(inputs):
    a, b = inputs.split(",")
    return float(a) + float(b)


# STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION 의 chat 은 chat_model에 최적화 되었다는 뜻임.
agent = initialize_agent(
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    tools=[
        Tool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. Use this tool by sending a pair of number separated by a comma.\nExample: 1,2",
        )
    ],
)

prompt = "Please calculate the total sum of the following numbers. Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"


agent.invoke(prompt)

{'input': 'Please calculate the total sum of the following numbers. Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': '$5273.38'}